In [ ]:
import heapq
import math

def stateful_dijkstra(graph, start_node, objectives):
    """
    Finds the shortest paths from a start node, considering multiple objectives.
    
    Args:
        graph (dict): Adjacency list representation of the graph.
                      e.g., {'A': [('B', {'cost': 10, 'time': 5})]}
        start_node: The starting node for the path.
        objectives (list): A list of objective names, e.g., ['cost', 'time'].
        
    Returns:
        dict: A dictionary mapping each node to its best path vector.
    """
    # Dictionary to store the best vector of objectives for each node
    # Initialize with infinity for all nodes except the start
    best_paths = {node: [math.inf] * len(objectives) for node in graph}
    best_paths[start_node] = [0] * len(objectives)

    # Priority queue: (current_objective_vector, current_node)
    pq = [(tuple([0] * len(objectives)), start_node)]

    while pq:
        current_objectives_tuple, current_node = heapq.heappop(pq)
        current_objectives = list(current_objectives_tuple)

        # Check if we have already found a better path
        if any(current_objectives[i] > best_paths[current_node][i] for i in range(len(objectives))):
            continue

        for neighbor, edge_data in graph.get(current_node, []):
            new_objectives = [current_objectives[i] + edge_data[objectives[i]] for i in range(len(objectives))]

            # Simple domination check for single-path Dijkstra
            if all(new_objectives[i] < best_paths[neighbor][i] for i in range(len(objectives))):
                best_paths[neighbor] = new_objectives
                heapq.heappush(pq, (tuple(new_objectives), neighbor))

    return best_paths

# Example usage of stateful Dijkstra
# graph = {
#     'A': [('B', {'cost': 10, 'time': 5}), ('C', {'cost': 1, 'time': 20})],
#     'B': [('D', {'cost': 5, 'time': 3})],
#     'C': [('D', {'cost': 1, 'time': 10})],
#     'D': []
# }
# shortest_paths = stateful_dijkstra(graph, 'A', ['cost', 'time'])
# print("Shortest paths from A:", shortest_paths)

In [ ]:
def held_karp_tsp(dist_matrix):
    """
    Solves the Traveling Salesperson Problem using the Held-Karp algorithm (Bitmask DP).
    
    Args:
        dist_matrix (list of lists): A square matrix where dist_matrix[i][j] is the distance
                                     between city i and city j.
                                     
    Returns:
        float: The minimum total cost of the tour.
        list: The optimal tour path.
    """
    num_cities = len(dist_matrix)
    # dp[mask][last_city]
    dp = {}
    path = {}

    # Base case: start at city 0, visiting only itself
    dp[(1 << 0, 0)] = 0

    # Iterate through all subsets of cities (represented by masks)
    for mask in range(1, 1 << num_cities):
        for last_city in range(num_cities):
            # If the last_city is in the current mask
            if (mask >> last_city) & 1:
                prev_mask = mask ^ (1 << last_city)
                
                # If this is not the first city visited
                if prev_mask != 0:
                    min_cost = float('inf')
                    prev_city_best = -1
                    
                    for prev_city in range(num_cities):
                        if (prev_mask >> prev_city) & 1:
                            if (prev_mask, prev_city) in dp:
                                cost = dp[(prev_mask, prev_city)] + dist_matrix[prev_city][last_city]
                                if cost < min_cost:
                                    min_cost = cost
                                    prev_city_best = prev_city
                    
                    if min_cost != float('inf'):
                        dp[(mask, last_city)] = min_cost
                        path[(mask, last_city)] = prev_city_best

    # Find the minimum cost to return to the start city (city 0)
    final_mask = (1 << num_cities) - 1
    min_tour_cost = float('inf')
    last_city_best = -1
    for last_city in range(1, num_cities):
        cost = dp.get((final_mask, last_city), float('inf')) + dist_matrix[last_city][0]
        if cost < min_tour_cost:
            min_tour_cost = cost
            last_city_best = last_city

    # Reconstruct the optimal path
    optimal_path = [0]
    current_mask = final_mask
    current_city = last_city_best
    
    while current_city != 0:
        optimal_path.insert(1, current_city)
        prev_city = path[(current_mask, current_city)]
        current_mask = current_mask ^ (1 << current_city)
        current_city = prev_city
        
    return min_tour_cost, optimal_path

# Example usage of Held-Karp
# dist_matrix = [[0, 10, 15, 20], [10, 0, 35, 25], [15, 35, 0, 30], [20, 25, 30, 0]]
# cost, path = held_karp_tsp(dist_matrix)
# print(f"Held-Karp TSP - Min Cost: {cost}, Path: {path}")

In [ ]:
def pareto_pruning(solutions):
    """
    Performs Pareto pruning on a list of solutions with multiple objectives.
    
    Args:
        solutions (list of tuples): Each tuple represents a solution, e.g.,
                                    (objective_vector, path_data).
                                    objective_vector is a tuple of costs (e.g., (cost, time)).
                                    
    Returns:
        list of tuples: A list containing only the non-dominated solutions.
    """
    if not solutions:
        return []

    # Sort solutions to optimize the pruning process
    solutions.sort()
    
    pareto_front = []
    
    for current_solution in solutions:
        is_dominated = False
        
        # Check if the current solution is dominated by any solution already in the Pareto front
        for pareto_solution in pareto_front:
            # A solution is dominated if all of its objective values are >=
            # than a Pareto solution's values, AND at least one is strictly >.
            if all(current_solution[0][i] >= pareto_solution[0][i] for i in range(len(current_solution[0]))) and \
               any(current_solution[0][i] > pareto_solution[0][i] for i in range(len(current_solution[0]))):
                is_dominated = True
                break
        
        if not is_dominated:
            # If the current solution is not dominated, it's a new member of the front.
            # We also need to remove any existing solutions in the front that it now dominates.
            new_pareto_front = []
            for existing_solution in pareto_front:
                # Keep existing solutions that are not dominated by the new one
                if not (all(existing_solution[0][i] >= current_solution[0][i] for i in range(len(existing_solution[0]))) and \
                        any(existing_solution[0][i] > current_solution[0][i] for i in range(len(existing_solution[0])))):
                    new_pareto_front.append(existing_solution)
            
            new_pareto_front.append(current_solution)
            pareto_front = new_pareto_front

    return pareto_front

# Example usage of Pareto Pruning
# A list of solutions with (cost, time) objectives
# solutions = [((10, 5), 'path1'), ((8, 7), 'path2'), ((12, 4), 'path3'), ((9, 6), 'path4')]
# non_dominated = pareto_pruning(solutions)
# print("Non-dominated solutions:", non_dominated)